# GapsTagger

In [1]:
from estnltk import Text
from estnltk.layer import Layer
from estnltk.spans import Span
from estnltk.taggers.gaps_tagging.gaps_tagger import GapsTagger

In [2]:
text = Text('Üks kaks kolm neli viis kuus seitse.')
layer_1 = Layer('test_1')
layer_1.add_span(Span(4, 8))
layer_1.add_span(Span(9, 13))
layer_1.add_span(Span(24, 28))
text['test_1'] = layer_1

layer_2 = Layer('test_2')
layer_2.add_span(Span(4, 8))
layer_2.add_span(Span(9, 18))
layer_2.add_span(Span(35, 36))
text['test_2'] = layer_2

## Example 1

In [3]:
gaps_tagger = GapsTagger('simple_gaps', ['test_1', 'test_2'])
gaps_tagger.tag(text)
text.simple_gaps

SL[Span(Üks , {}),
Span( , {}),
Span( viis , {}),
Span( seitse, {})]

The following illustrates examples 1 and 2.

    text:           'Üks kaks kolm neli viis kuus seitse.'
    test_1:             'kaks'kolm'         'kuus'      
    test_2:             'kaks'kolm neli'               '.'
    simple_gaps:    'Üks '  ' '       ' viis '  ' seitse'
    gaps:           'Üks'              'viis'    'seitse'

## Example 2

In [4]:
def trim(text:str, s:int, e:int):
    t_0 = text[s:e]
    t_1 = t_0.strip()
    i = t_0.find(t_1)
    return (s+i, s+i+len(t_1))

def decorator(text:str):
    return {'gap_length':len(text)}

gaps_tagger = GapsTagger(layer_name='gaps',
                         input_layers=['test_1', 'test_2'],
                         trim=trim, decorator=decorator,
                         attributes=['gap_length'])
gaps_tagger.tag(text)
text.gaps

SL[Span(Üks, {'gap_length': 3}),
Span(viis, {'gap_length': 4}),
Span(seitse, {'gap_length': 6})]

In [5]:
gaps_tagger

In [6]:
from estnltk.taggers.merge_tagging.merge_tagger import MergeTagger

merge_tagger = MergeTagger('merged', input_layers=['test_1', 'test_2'])
merge_tagger.tag(text)

In [7]:
text

Text(text="Üks kaks kolm neli viis kuus seitse.")

In [8]:
text.merged

SL[Span(kaks, {}),
Span(kaks, {}),
Span(kolm, {}),
Span(kolm neli, {}),
Span(kuus, {}),
Span(., {})]

In [9]:
text.test_1

SL[Span(kaks, {}),
Span(kolm, {}),
Span(kuus, {})]

In [10]:
text.test_2

SL[Span(kaks, {}),
Span(kolm neli, {}),
Span(., {})]